In [1]:
import pandas as pd
import numpy as np
import os
import nltk
import nltk.tokenize
import nltk.tag
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#from nltk.corpus import stopwords
import statistics as stat
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [31]:
def int_punct(df): 
    res_vi = []
    res_pv = []
    res_ti = []
    for k in range(len(df.axes[0])): 
        a = df['Main_Sentence_x'][k]
        v = 0
        p = 0
        t = 0
        if pd.isna(a)==False: 
            for c in a: 
                if c==',': 
                    v = v+1
                if c==';':
                    p = p+1
                if c==' - ': # espace pour ne pas prendre en compte les mots composes 
                    t = t+1
        s = v+p+t
        if s==0: 
            res_vi.append(None)
            res_pv.append(None)
            res_ti.append(None)
        else:     
            res_vi.append(v/s*100)
            res_pv.append(p/s*100)
            res_ti.append(t/s*100)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Virgules_pct':res_vi,
                        'Point_virgules_pct':res_pv,
                        'Tirets_pct':res_ti})
    return(res)    

# Calcul des % pour la ponctuation externe aux phrases: 
def ext_punct(df): 
    res_dec = []
    res_int = []
    res_exc = []
    for k in range(len(df.axes[0])): 
        a = df['Main_Sentence_x'][k]
        d = 0
        i = 0
        e = 0
        if pd.isna(a)==False: 
            for c in a: 
                if c=='.': 
                    d = d+1
                if c=='?':
                    i = i+1
                if c=='!': 
                    e = e+1
        s = d+i+e
        if s==0: 
            res_dec.append(None)
            res_int.append(None)
            res_exc.append(None)
        else:     
            res_dec.append(d/s*100)
            res_int.append(i/s*100)
            res_exc.append(e/s*100)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Declaratives_pct':res_dec,
                        'Interrogatives_pct':res_int,
                        'Exclamatives_pct':res_exc})
    return(res)
    
# Calcul des % des differents types de mots dans chaque critique: 
def type_mots(df):
    # Les types de mots recherches: 
    adv = ['RB','RBR','RBS']
    nn = ['NN','NNS','NNP','NNPS']
    vb = ['VB','VBD','VBG','VBN','VBP','VBZ']
    adj = ['JJ','JJR']
    ads = ['JJS']
    # Listes vides pour stocker les resultats
    res_mots = []
    res_adv = []
    res_nn = []
    res_vb = []
    res_adj = []
    res_ads = []
    for k in range(len(df.axes[0])):
    #for k in range(10):
        t = df['Main_Sentence_x'][k]
        # Set counters to 0: 
        ca = 0
        cn = 0
        cv = 0
        cj = 0
        cs = 0
        if pd.isna(t)==False: 
            t_tok = nltk.word_tokenize(t)
            res_mots.append(len(t_tok))
            if len(t_tok)==0: # ie si pas de texte de critique
                res_adv.append(None)
                res_nn.append(None)
                res_vb.append(None)
                res_adj.append(None)
                res_ads.append(None)
            else: 
                t_tag = nltk.pos_tag(t_tok)
                for x in t_tag: 
                    if x[1] in adv: 
                        ca = ca+1
                    if x[1] in nn: 
                        cn = cn+1
                    if x[1] in vb: 
                        cv = cv+1
                    if x[1] in adj: 
                        cj = cj+1
                    if x[1] in ads: 
                        cs = cs+1
                res_adv.append(ca/len(t_tok))
                res_nn.append(cn/len(t_tok))
                res_vb.append(cv/len(t_tok))
                res_adj.append(cj/len(t_tok))
                res_ads.append(cs/len(t_tok))
        else: 
            res_mots.append(None)
            res_adv.append(None)
            res_nn.append(None)
            res_vb.append(None)
            res_adj.append(None)
            res_ads.append(None)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Nb_mots':res_mots,
                        'Adverbes':res_adv,
                        'Noms':res_nn,
                        'Verbes':res_vb,
                        'Adjectifs':res_adj,
                        'Superlatifs':res_ads})
    return res

# 15 mots les plus frequents, selon le type de mots: 
# ty = type de mot dont on souhaite obtenir le top 15
# Valeurs possibles: 'nom', 'adj', 'adv', 'sup', 'vb'
def top_15(df,ty): 
    # Def. des tags a rechercher: 
    if ty=='nom': tg = ['NN','NNS','NNP','NNPS']
    if ty=='adj': tg = ['JJ','JJR']
    if ty=='sup': tg = ['JJS']
    if ty=='adv': tg = ['RB','RBR','RBS']
    if ty=='vb': tg = ['VB','VBD','VBG','VBN','VBP','VBZ']
    l = {} # dictionnaire qui prendra en cle les mots et en valeur leur nb d'occurrences 
    for k in range(len(df.axes[0])):
        t = df['Main_Sentence_x'][k]
        if pd.isna(t)==False: 
            t = t.lower()
            t_tok = nltk.word_tokenize(t)
            if len(t_tok)!=0:
                t_tag = nltk.pos_tag(t_tok)
                for x in t_tag: 
                    if x[1] in tg: 
                        if x[0] in l: 
                            l[x[0]] = l[x[0]]+1
                        else: 
                            l[x[0]]=1
    res = sorted(l.items(), key=lambda x: x[1]) # liste de couples
    # Retirer les 'parasites': 
    pb = ['"','“','’','—','”','t','n','‘','s']
    res = [x for x in res if x[0] not in pb]        
    res15 = res[len(res)-16:len(res)-1]
    return res15

# % verbes conjugues au passe / present / futur: 
def tps_vb(df): 
    pr = ['VBP','VBZ','VBG']
    ps = ['VBD','VBN']
    # VB: cas particulier (present / futur)
    res_pr = []
    res_ps = []
    res_fu = []
    for k in range(len(df.axes[0])):
        t = df['Main_Sentence_x'][k]
        cpr = 0
        cps = 0
        cfu = 0
        if pd.isna(t)==False: 
            t_tok = nltk.word_tokenize(t)
            if len(t_tok)==0: # ie si pas de texte de critique
                res_pr.append(None)
                res_ps.append(None)
                res_fu.append(None)
            else: 
                t_tag = nltk.pos_tag(t_tok)
                for j in range(len(t_tag)): 
                    if t_tag[j][1] in pr: 
                        cpr = cpr+1
                    if t_tag[j][1] in ps: 
                        cps = cps+1
                    if t_tag[j][1]=='VB': 
                        if j!=0: 
                            if t_tag[j-1][0]=='will': 
                                cfu = cfu+1
                        else: 
                            cpr = cpr+1
                res_pr.append(cpr)
                res_ps.append(cps)
                res_fu.append(cfu)
        else: 
            res_pr.append(None)
            res_ps.append(None)
            res_fu.append(None)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Passe':res_ps,
                        'Present':res_pr,
                        'Futur':res_fu})
    return res

# Pronoms personnels utilises:
def pronoms_perso(df): 
    # Pronoms que l'on veut compter: 
    i = ['I','me','my','mine']
    you = ['you','yours','your']
    he = ['he','she','him','his','her','hers']
    we = ['we','us','our','ours']
    they = ['they','them','their','theirs']
    res_i = [] 
    res_you = [] 
    res_he = [] 
    res_we = [] 
    res_they = [] 
    for k in range(len(df.axes[0])):
        t = df['Main_Sentence_x'][k]
        ci = 0
        cy = 0
        ch = 0
        cw = 0
        ct = 0
        if pd.isna(t)==False: 
            t_tok = nltk.word_tokenize(t)
            if len(t_tok)==0: # ie si pas de texte de critique
                res_i.append(None)
                res_you.append(None)
                res_he.append(None)
                res_we.append(None)
                res_they.append(None)
            else: 
                t_tag = nltk.pos_tag(t_tok)
                for x in t_tag: 
                    if x[1]=='PRP' or x[1]=='PRP$':
                        if x[0] in i: 
                            ci = ci+1
                        if x[0] in you: 
                            cy = cy+1
                        if x[0] in he: 
                            ch = ch+1
                        if x[0] in we: 
                            cw = cw+1
                        if x[0] in they: 
                            ct = ct+1
                res_i.append(ci)
                res_you.append(cy)
                res_he.append(ch)
                res_we.append(cw)
                res_they.append(ct)
        else: 
            res_i.append(None)
            res_you.append(None)
            res_he.append(None)
            res_we.append(None)
            res_they.append(None)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        '1e_pers_s':res_i,
                        '2e_pers':res_you,
                        '3e_pers_s':res_he,
                        '1e_pers_p':res_we,
                        '3e_pers_p':res_they})
    return res


In [39]:
df_int_zone = int_punct(df)
df_ext_zone = ext_punct(df)
df_type_mots_zone = type_mots(df)
df_tps_vb_zone = tps_vb(df)
df_pronoms_perso_zone = pronoms_perso(df)

In [41]:
df_int_zone.head(3)

,ID,Virgules_pct,Point_virgules_pct,Tirets_pct
0,0,100.0,0.0,0.0
1,1,100.0,0.0,0.0
2,2,100.0,0.0,0.0


In [42]:
df_ext_zone.head(3)

,ID,Declaratives_pct,Interrogatives_pct,Exclamatives_pct
0,0,100.0,0.0,0.0
1,1,100.0,0.0,0.0
2,2,100.0,0.0,0.0


In [43]:
df_type_mots_zone.head(3)

,ID,Nb_mots,Adverbes,Noms,Verbes,Adjectifs,Superlatifs
0,0,141,0.021277,0.319149,0.127660,0.085106,0.007092
1,1,88,0.034091,0.261364,0.215909,0.068182,0.000000
2,2,93,0.032258,0.268817,0.118280,0.118280,0.000000


In [44]:
df_tps_vb_zone.head(3)

,ID,Passe,Present,Futur
0,0,8,6,0
1,1,10,8,0
2,2,3,8,0


In [45]:
df_pronoms_perso_zone.head(3)

,ID,1e_pers_s,2e_pers,3e_pers_s,1e_pers_p,3e_pers_p
0,0,1,1,0,0,0
1,1,0,0,2,0,1
2,2,0,0,0,0,1


In [46]:
df = pd.read_excel("zoning_v2.xlsx")
df.head(3)

,Unnamed: 0,Main_Sentence_x,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Main_Sentence_y,Sentence_Type,Type_word_k,Type_word_v
0,0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ..."
1,1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[('otto', 0.3578513897480544), ('lloyd', 0.357...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[4, 7, 5, 9, 2, 11, 12, 1, 4, 8, 13, 2, 9]"
2,2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[('theatrical', 0.28284610378675523), ('women'...",481,0,0,4.95,The clock is turned back in every case this we...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[9, 12, 3, 4, 4, 15, 16, 1, 1, 3, 8, 6, 1, 7]"


In [47]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [48]:
df.head(3)

,Main_Sentence_x,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Main_Sentence_y,Sentence_Type,Type_word_k,Type_word_v
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ..."
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[('otto', 0.3578513897480544), ('lloyd', 0.357...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[4, 7, 5, 9, 2, 11, 12, 1, 4, 8, 13, 2, 9]"
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[('theatrical', 0.28284610378675523), ('women'...",481,0,0,4.95,The clock is turned back in every case this we...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[9, 12, 3, 4, 4, 15, 16, 1, 1, 3, 8, 6, 1, 7]"


In [49]:
df = pd.concat([df, df_int_zone,df_ext_zone,df_type_mots_zone,df_tps_vb_zone,df_pronoms_perso_zone], axis=1)
display(df.head(3))
display(df.shape)

,Main_Sentence_x,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Main_Sentence_y,Sentence_Type,Type_word_k,Type_word_v,ID,Virgules_pct,Point_virgules_pct,Tirets_pct,ID,Declaratives_pct,Interrogatives_pct,Exclamatives_pct,ID,Nb_mots,Adverbes,Noms,Verbes,Adjectifs,Superlatifs,ID,Passe,Present,Futur,ID,1e_pers_s,2e_pers,3e_pers_s,1e_pers_p,3e_pers_p
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ...",0,100.0,0.0,0.0,0,100.0,0.0,0.0,0,141,0.021277,0.319149,0.127660,0.085106,0.007092,0,8,6,0,0,1,1,0,0,0
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[('otto', 0.3578513897480544), ('lloyd', 0.357...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[4, 7, 5, 9, 2, 11, 12, 1, 4, 8, 13, 2, 9]",1,100.0,0.0,0.0,1,100.0,0.0,0.0,1,88,0.034091,0.261364,0.215909,0.068182,0.000000,1,10,8,0,1,0,0,2,0,1
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[('theatrical', 0.28284610378675523), ('women'...",481,0,0,4.95,The clock is turned back in every case this we...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[9, 12, 3, 4, 4, 15, 16, 1, 1, 3, 8, 6, 1, 7]",2,100.0,0.0,0.0,2,100.0,0.0,0.0,2,93,0.032258,0.268817,0.118280,0.118280,0.000000,2,3,8,0,2,0,0,0,0,1


(241, 79)

In [50]:
df_review = pd.read_excel("df_review.xlsx")
display(df_review.head(3))
display(df_review.shape)

,Blog,Publication Date,Reviewer,Title of the Play,Playwright,Theatre,Review with Zoning,Intro-Purple,Red-Plot,Blue-General,Green-Details,Orange-Perf,Borwn-Structure,Yellow-Conclusion,Latitude,Longitude,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Start_Review_1,End_Review_1,Start_Review_2,End_Review_2,Start_Review_3,End_Review_3,Nbr_of_Sentences,Length_Start_Count_Words_1,Length_End_Count_Words_1,Length_Start_Count_Words_2,Length_End_Count_Words_2,Length_Start_Count_Words_3,Length_End_Count_Words_3,Question_Mark_Start_1,Question_Mark_End_1,Question_Mark_Start_2,Question_Mark_End_2,Question_Mark_Start_3,Question_Mark_End_3,Exclamation_Mark_Start_1,Exclamation_Mark_End_1,Exclamation_Mark_Start_2,Exclamation_Mark_End_2,Exclamation_Mark_Start_3,Exclamation_Mark_End_3,Word_Average_Start_1,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3,Main_Sentence,Sentence_Type
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38,Sigmund Freud was a great admirer of the Austr...,1
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey...,[' And the special effects -they include singi...,25,11,7,30,35,51,60,0,0,0,0,0,0,0,0,0,0,0,0,4.27,3.00,5.00,4.88,4.82,4.81,Love never dies has suffered a rocky journey t...,1
2,SUNDAY EXPRESS,28.03.10,Mark Shenton,THE WHITE GUARD,Andrew Upton,Lyttelton Theatre,The clock is turned back in every case this we...,The clock is turned back in every case this we...,"The sole woman, Elena (a forceful Justine Mitc...",NaN,NaN,NaN,NaN,"Written across a broad, epic canvas and given...",51.507219,-0.114367,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.060,0.809,0.131,0.9052,0.195650,0.377885,0.07,0.55,0.10,0.10,0.17,"[('davies', 0.294100664407846), ('forces', 0.2...",n.a,n.a,n.a,n.a,n.a,n.a,6,1,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,1.00,1.00,1.00,1.00,The clock is turned back in every case this we...

(350, 93)

In [51]:
df_review = df_review.add_prefix('Review - ')

In [52]:
df_review.head(3)

,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38,Sigmund Freud was a great admirer of the Austr...,1
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey

In [53]:
df_review = df_review.drop_duplicates(subset='Review - Main_Sentence', keep="last").reset_index(drop=True)
display(df_review.head(3))
display(df_review.shape)

,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38,Sigmund Freud was a great admirer of the Austr...,1
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey

(228, 93)

In [54]:
df_zone = df
df_zone = df_zone.add_prefix('Zone - ')

In [55]:
display(df_zone.head(3))
display(df_zone.shape)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ...",0,100.0,0.0,0.0,0,100.0,0.0,0.0,0,141,0.021277,0.319149,0.127660,0.085106,0.007092,0,8,6,0,0,1,1,0,0,0
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[('otto', 0.3578513897480544), ('lloyd', 0.357...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[4, 7, 5, 9, 2, 11, 12, 1, 4, 8, 13, 2, 9]",1,100.0,0.0,0.0,1,100.0,0.0,0.0,1,88,0.034091,0.261364,0.215909,0.068182,0.000000,1,10,8,0,1,0,0,2,0,1
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[('theatrical', 0.28284610378675523), ('women'...",481,0,0,4.95,The clock is turned back in every case this we...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[9, 12, 3, 4, 4, 15, 16, 1, 1, 3, 8, 6, 1, 7]",2,100.0,0.0,0.0,2,100.0,0.0,0.0,2,93,0.032258,0.268817,0.118280,0.118280,0.000000,2,3,8,0,2,0,0,0,0,1


(241, 79)

In [56]:
df_review = df_review[df_review["Review - Main_Sentence"].isna() == False].reset_index(drop = True)
display(df_review.head(3))
display(df_review.shape)

,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38,Sigmund Freud was a great admirer of the Austr...,1
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey

(227, 93)

In [57]:
df_zone.shape

(241, 79)

In [58]:
df_zone = df_zone.drop_duplicates(subset='Zone - Main_Sentence_x', keep="last").reset_index(drop=True)
display(df_zone.head(3))
display(df_zone.shape)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ...",0,100.0,0.0,0.0,0,100.0,0.0,0.0,0,141,0.021277,0.319149,0.127660,0.085106,0.007092,0,8,6,0,0,1,1,0,0,0
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[('otto', 0.3578513897480544), ('lloyd', 0.357...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[4, 7, 5, 9, 2, 11, 12, 1, 4, 8, 13, 2, 9]",1,100.0,0.0,0.0,1,100.0,0.0,0.0,1,88,0.034091,0.261364,0.215909,0.068182,0.000000,1,10,8,0,1,0,0,2,0,1
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[('theatrical', 0.28284610378675523), ('women'...",481,0,0,4.95,The clock is turned back in every case this we...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[9, 12, 3, 4, 4, 15, 16, 1, 1, 3, 8, 6, 1, 7]",2,100.0,0.0,0.0,2,100.0,0.0,0.0,2,93,0.032258,0.268817,0.118280,0.118280,0.000000,2,3,8,0,2,0,0,0,0,1


(227, 79)

In [59]:
df_final_v1 = pd.merge(df_zone, df_review, how='left', left_on='Zone - Main_Sentence_x', right_on='Review - Main_Sentence')
display(df_final_v1.head(3))
display(df_final_v1.shape)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ..."

(227, 172)

In [60]:
df_final_v1.shape

(227, 172)

In [61]:
display(df_final_v1.head(3))
display(df_final_v1.shape)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ..."

(227, 172)

In [63]:
list_start_index = []
for i in range(0,len(df_final_v1["Review - Review with Zoning"])):
   list_start_index.append(df_final_v1["Review - Review with Zoning"][i].find(df_final_v1["Zone - Main_Sentence_x"][i]))


In [64]:
df_final_v1["Zone - Start Index"] = list_start_index

In [65]:
df_final_v2 = df_final_v1.copy()

In [66]:
df_final_v2.head(3)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type,Zone - Start Index
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', '...","[15, 19, 4, 6, 1, 13, 22, 3, 2,

In [67]:
df_final_v2.shape

(227, 173)

In [68]:
len(df_final_v2["Zone - Main_Sentence_x"][0])

761

In [69]:
list_count_character_zone = []
list_count_character_review = []

for i in range(0,len(df_final_v2["Zone - Main_Sentence_x"])):
    list_count_character_zone.append(len(df_final_v2["Zone - Main_Sentence_x"][i]))
    list_count_character_review.append(len(df_final_v2["Review - Review with Zoning"][i]))
    

In [70]:
df_final_v2["Zone - Count Character"] = list_count_character_zone
df_final_v2["Review - Count Character"] = list_count_character_review

In [71]:
list_count_word_zone = []
list_count_word_review = []

for i in range(0,len(df_final_v2["Zone - Main_Sentence_x"])):
    list_count_word_zone.append(len(df_final_v2["Zone - Main_Sentence_x"][i]))
    list_count_word_review.append(len(df_final_v2["Review - Review with Zoning"][i]))
    

In [72]:
df_final_v2["Zone - Count Word"] = list_count_word_zone
df_final_v2["Review - Count Word"] = list_count_word_review

In [73]:
display(df_final_v2.head(3))
display(df_final_v2.shape)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Review - Sentence_Type,Zone - Start Index,Zone - Count Character,Review - Count Character,Zone - Count Word,Review - Count Word
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,

(227, 177)

In [74]:
df_final_v2 = df_final_v2.rename(columns={'Review - Sentence_Type': 'Target_Variable_Sentence_Type'})
df_final_v2.head(3)

,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Review - Neu,Review - Pos,Review - Compound,Review - Polarity,Review - Subjectivity,Review - Angry,Review - Fear,Review - Happy,Review - Sad,Review - Surpise,Review - Top_10_Keywords,Review - Start_Review_1,Review - End_Review_1,Review - Start_Review_2,Review - End_Review_2,Review - Start_Review_3,Review - End_Review_3,Review - Nbr_of_Sentences,Review - Length_Start_Count_Words_1,Review - Length_End_Count_Words_1,Review - Length_Start_Count_Words_2,Review - Length_End_Count_Words_2,Review - Length_Start_Count_Words_3,Review - Length_End_Count_Words_3,Review - Question_Mark_Start_1,Review - Question_Mark_End_1,Review - Question_Mark_Start_2,Review - Question_Mark_End_2,Review - Question_Mark_Start_3,Review - Question_Mark_End_3,Review - Exclamation_Mark_Start_1,Review - Exclamation_Mark_End_1,Review - Exclamation_Mark_Start_2,Review - Exclamation_Mark_End_2,Review - Exclamation_Mark_Start_3,Review - Exclamation_Mark_End_3,Review - Word_Average_Start_1,Review - Word_Average_End_1,Review - Word_Average_Start_2,Review - Word_Average_End_2,Review - Word_Average_Start_3,Review - Word_Average_End_3,Review - Main_Sentence,Target_Variable_Sentence_Type,Zone - Start Index,Zone - Count Character,Review - Count Character,Zone - Count Word,Review - Count Word
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[('sexuality', 0.2538867748491288), ('harrower...",761,0,0,5.46,Sigmund Freud was a great admirer of the Au

In [75]:
df_final_v2.to_excel("zoning_final.xlsx",index= False)